### General Steps:
- [ ]  Sound source matching
- [ ]  Timestamp matching
- [ ]  Delay and significance calculation
- [ ]  Extract coordinates
- [ ]  Populate dataframe

======================================================================================================================
### Current Goal:
Formulate data frame for 1 min recording from 1:05 pm - 1:15pm, on March 25th. Single stationary source.

======================================================================================================================

#### Sound Source Matching

In [ ]:
'''
MS+LP: 
For 1 min recording starting at 1:05pm, given a window length of 1s and 4 arrays (16 channels). 
The number of cross correlation calculation is 60s/1s * (16 pick 2).
'''
import soundfile as sf
import math
import numpy as np
import scipy as signal
import matplotlib.pyplot as plt
import pandas as pd
import itertools

from functools import lru_cache

In [ ]:
# Functions
def truncate(sig1, sig2):
    '''
    This function truncates the longer signal and return two signals of the same length
    '''
    l1, l2 = len(sig1), len(sig2)
    if l1 <= l2:
        return sig1, sig2[0:l1]
    else:
        return sig1[0:l2], sig2
    
def padding(sig1, sig2):
    '''
    This function pads the shorter sequence to the length of longer sequence with 0
    '''
    length = np.maximum(len(sig1), len(sig2))
    l1, l2 = len(sig1), len(sig2)
    if l1 < length:
        return np.concatenate((sig1, np.linspace(0,0,length - l1))), sig2
    if l2 < length:
        return sig1, np.concatenate((sig2, np.linspace(0,0,length - l2)))

def prepadding(sig1, sig2, winDuration, Fs):
    l = winDuration * Fs
    N = len(sig1)    
    # Padding to have an interger number of windows
    if N%l != 0:
        padWidth = l-N%l
        sig1 = np.pad(sig1, (0, padWidth), 'constant', constant_values = 0)
        sig2 = np.pad(sig2, (0, padWidth), 'constant', constant_values = 0)
        
    numOfWindows = len(sig1)/l
    
    return sig1, sig2, numOfWindows

def whiten(sig):
    return sig/np.abs(sig)

def gcc_phat(sig1, sig2, fs=16000, max_tau=None, interp=1, window=True, windowName="hamming"):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT) method. In this modified
    function, sig1 and sig2 are garuanteed to have the same length
    '''
    # Make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
    # n1, n2 = sig1.shape[0], sig2.shape[0]
    n = sig1.shape[0]
    
    # Add window
    if window: 
        win1 = getattr(np, windowName)(n)
        win2 = getattr(np, windowName)(n)
        sig1 = sig1 * win1
        sig2 = sig2 * win2

    # Generalized Cross Correlation Phase Transform
    SIG1 = np.fft.rfft(sig1, n=n)
    SIG2 = np.fft.rfft(sig2, n=n) 
    W1 = whiten(SIG1)
    W2 = whiten(SIG2)
    
    R = W1 * np.conj(W2)
    
    cc = np.fft.irfft(R, n=(interp * n))

    # cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    # find max cross correlation index
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = np.float(shift / float(interp * fs))
    
    if(np.any(np.isnan(cc))): 
        tau = math.nan
    
    return tau, cc, R, SIG1, SIG2


def width_cc(cc, threshold=0.5):
    '''
    Calculate the width between the first sample and the last sample that are above the thresold.
    Threshold is precentage of the peak value
    '''
    cc_max = np.amax(cc)
    peak_index = np.where(cc == cc_max)
    mag_thr = threshold * cc_max
    width = 0
    for ii in range(len(cc)):
        if cc[ii] > mag_thr:
            width = width + 1
    return width

def significance(cc):
    '''
    Calculate the significance of each cross correlation sequence
    '''
    cc_mean = np.mean(cc)
    cc_std = np.std(cc)
    return (np.amax(cc) - cc_mean)/cc_std

#### Timestamp Matching